# Definitive Version

## Gather all the data

In [3]:
import fastf1
import pandas as pd
import os

#dim reduction preprocessing
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
import numpy as np

In [8]:
year = 2023
#Get the schedule for the year
year_schedule = fastf1.get_event_schedule(year)
print(year_schedule['RoundNumber'])

#Repeat for every race 
for round in year_schedule['RoundNumber'][1:]: #Ignore the first round because it's the testing event of the season, not an actual race

    #Create a dircetory where to store all the files
    dirname = '../data/'+str(round)+'_'+year_schedule['Location'][round]
    os.mkdir(dirname)

    #Get the race and load the data
    race = fastf1.get_session(year, year_schedule['Location'][round], 'R')
    race.load()

    #Laps
    race.laps.to_csv(dirname+'/laps.csv')
    #Results
    race.results.to_csv(dirname+'/results.csv')
    #Race info (unused)
    race.event.to_csv(dirname+'/race_info.csv')
    #Weahter data (unused)
    race.weather_data.to_csv(dirname+'/weather_data.csv')
    #Track status
    race.track_status.to_csv(dirname+'/track_status.csv')
    
    #Create a subdirectory for the car telemetries
    for i in race.car_data.keys():
        race.car_data[i].to_csv(dirname+'/'+str(i)+'_telemetry.csv')


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
Name: RoundNumber, dtype: int32


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req

## Compute PCA


In [4]:
year = 2023
year_schedule = fastf1.get_event_schedule(year)
for round in year_schedule['RoundNumber'][1:]:
       race = fastf1.get_session(year, year_schedule['Location'][round], 'R')
       race.load()

       dataset = race.laps
       cols_to_remove =['DriverNumber','PitOutTime','PitInTime','IsPersonalBest','FreshTyre','LapStartTime', 'Position', 'Deleted','DeletedReason','FastF1Generated','IsAccurate']
       dataset = dataset.drop(cols_to_remove, axis = 1)
       dataset['TrackStatus'] = dataset['TrackStatus'].astype('float') #I hate this
       rows_to_remove = dataset.index[dataset['TrackStatus'] != 1].tolist()
       dataset = dataset.drop(rows_to_remove, axis = 0)


       #drop rows with missing data because i cant think of something better
       dataset = dataset.dropna(axis=0)
       #drop trackStatus as well after i used i to remove the rows i wanted
       dataset = dataset.drop('TrackStatus', axis = 1)
       #Convert timedelta64 to milliseconds
       cols_to_change = ['Time', 'LapTime','Sector1Time',
              'Sector2Time', 'Sector3Time', 'Sector1SessionTime',
              'Sector2SessionTime', 'Sector3SessionTime','LapStartDate']
       dataset[cols_to_change] = dataset[cols_to_change].astype(np.int64)/ int(1e6)
       dataset =dataset.reset_index()
       print(dataset.info())
       
       cols = ['Time', 'LapTime', 'LapNumber', 'Stint', 'Sector1Time',
              'Sector2Time', 'Sector3Time', 'Sector1SessionTime',
              'Sector2SessionTime', 'Sector3SessionTime', 'SpeedI1', 'SpeedI2',
              'SpeedFL', 'SpeedST', 'TyreLife', 'LapStartDate']

       scaler= StandardScaler()
       scaled = scaler.fit_transform(dataset[cols])

       pca = PCA(n_components = 2)
       components = pca.fit_transform(scaled)
       principal_components = pca.components_
       df_components = pd.DataFrame(data = components, columns=['PC1','PC2'])

       df_components['Driver'] = dataset['Driver']
       df_components['Team'] = dataset['Team']
       df_components[cols] = dataset[cols]
       print(df_components)

       dirname = '../data/'+str(round)+'_'+year_schedule['Location'][round]
       df_components.to_csv(dirname+'/'+'PCA.csv')

req         WARNING 	DEFAULT CACHE ENABLED! (2.34 GB) C:\Users\perot\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '

<class 'fastf1.core.Laps'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               599 non-null    int64  
 1   Time                599 non-null    float64
 2   Driver              599 non-null    object 
 3   LapTime             599 non-null    float64
 4   LapNumber           599 non-null    float64
 5   Stint               599 non-null    float64
 6   Sector1Time         599 non-null    float64
 7   Sector2Time         599 non-null    float64
 8   Sector3Time         599 non-null    float64
 9   Sector1SessionTime  599 non-null    float64
 10  Sector2SessionTime  599 non-null    float64
 11  Sector3SessionTime  599 non-null    float64
 12  SpeedI1             599 non-null    float64
 13  SpeedI2             599 non-null    float64
 14  SpeedFL             599 non-null    float64
 15  SpeedST             599 non-null    float64
 16  Compound           

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


<class 'fastf1.core.Laps'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               605 non-null    int64  
 1   Time                605 non-null    float64
 2   Driver              605 non-null    object 
 3   LapTime             605 non-null    float64
 4   LapNumber           605 non-null    float64
 5   Stint               605 non-null    float64
 6   Sector1Time         605 non-null    float64
 7   Sector2Time         605 non-null    float64
 8   Sector3Time         605 non-null    float64
 9   Sector1SessionTime  605 non-null    float64
 10  Sector2SessionTime  605 non-null    float64
 11  Sector3SessionTime  605 non-null    float64
 12  SpeedI1             605 non-null    float64
 13  SpeedI2             605 non-null    float64
 14  SpeedFL             605 non-null    float64
 15  SpeedST             605 non-null    float64
 16  Compound           

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '14', '18', '11', '4', '27', '81', '24', '22', '77', '55', '10', '31', '21', '2', '20', '63', '23', '16']
core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


...

<class 'fastf1.core.Laps'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               729 non-null    int64  
 1   Time                729 non-null    float64
 2   Driver              729 non-null    object 
 3   LapTime             729 non-null    float64
 4   LapNumber           729 non-null    float64
 5   Stint               729 non-null    float64
 6   Sector1Time         729 non-null    float64
 7   Sector2Time         729 non-null    float64
 8   Sector3Time         729 non-null    float64
 9   Sector1SessionTime  729 non-null    float64
 10  Sector2SessionTime  729 non-null    float64
 11  Sector3SessionTime  729 non-null    float64
 12  SpeedI1             729 non-null    float64
 13  SpeedI2             729 non-null    float64
 14  SpeedFL             729 non-null    float64
 15  SpeedST             729 non-null    float64
 16  Compound           

req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '31', '18', '55', '44', '63', '14', '81', '10', '23', '20', '3', '24', '2', '77', '22', '27', '4']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_

<class 'fastf1.core.Laps'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               556 non-null    int64  
 1   Time                556 non-null    float64
 2   Driver              556 non-null    object 
 3   LapTime             556 non-null    float64
 4   LapNumber           556 non-null    float64
 5   Stint               556 non-null    float64
 6   Sector1Time         556 non-null    float64
 7   Sector2Time         556 non-null    float64
 8   Sector3Time         556 non-null    float64
 9   Sector1SessionTime  556 non-null    float64
 10  Sector2SessionTime  556 non-null    float64
 11  Sector3SessionTime  556 non-null    float64
 12  SpeedI1             556 non-null    float64
 13  SpeedI2             556 non-null    float64
 14  SpeedFL             556 non-null    float64
 15  SpeedST             556 non-null    float64
 16  Compound           

req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '11', '4', '81', '14', '22', '44', '18', '3', '31', '10', '23', '27', '2', '24', '55', '77', '20']


<class 'fastf1.core.Laps'>
RangeIndex: 845 entries, 0 to 844
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               845 non-null    int64  
 1   Time                845 non-null    float64
 2   Driver              845 non-null    object 
 3   LapTime             845 non-null    float64
 4   LapNumber           845 non-null    float64
 5   Stint               845 non-null    float64
 6   Sector1Time         845 non-null    float64
 7   Sector2Time         845 non-null    float64
 8   Sector3Time         845 non-null    float64
 9   Sector1SessionTime  845 non-null    float64
 10  Sector2SessionTime  845 non-null    float64
 11  Sector3SessionTime  845 non-null    float64
 12  SpeedI1             845 non-null    float64
 13  SpeedI2             845 non-null    float64
 14  SpeedFL             845 non-null    float64
 15  SpeedST             845 non-null    float64
 16  Compound           

# Compute t-SNE

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
def scatter(x, colors,n_classes,Title):
    # We choose a color palette with seaborn, a color for each class
    palette = np.array(sns.color_palette("hls", n_classes))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=20,c =palette[colors.astype(int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')
    xtext = -30 
    ytext = 50

    txts = []
    txt = ax.text(xtext, ytext, Title, fontsize=20)
    txts.append(txt)

    # We add the labels for each digit    
    for i in range(n_classes):
        # Position of each label.
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=16)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
    return f, ax, sc, txts

In [4]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import fastf1

year = 2023
year_schedule = fastf1.get_event_schedule(year)
for round in year_schedule['RoundNumber'][1:]:
       race = fastf1.get_session(year, year_schedule['Location'][round], 'R')
       race.load()

       dataset = race.laps
       cols_to_remove =['Time', 'DriverNumber','PitOutTime','PitInTime', 'Sector1SessionTime', 
                        'Sector2SessionTime', 'Sector3SessionTime', 'IsPersonalBest',
                        'FreshTyre','LapStartTime','LapStartDate', 'Deleted',
                        'DeletedReason','FastF1Generated','IsAccurate']
       dataset = dataset.drop(cols_to_remove, axis = 1)
       dataset['TrackStatus'] = dataset['TrackStatus'].astype('float') #I hate this
       #drop rows where track status is not 1, i.e. the lap is not a 'racing' lap
       rows_to_remove = dataset.index[dataset['TrackStatus'] != 1].tolist()
       dataset = dataset.drop(rows_to_remove, axis = 0)


       #drop rows with missing data because i cant think of something better
       dataset = dataset.dropna(axis=0)
       #drop trackStatus as well after i used i to remove the rows i wanted
       dataset = dataset.drop('TrackStatus', axis = 1)
       #Convert timedelta64 to milliseconds
       cols_to_change = ['LapTime','Sector1Time',
              'Sector2Time', 'Sector3Time']
       dataset[cols_to_change] = dataset[cols_to_change].astype(np.int64)/ int(1e6)
       dataset =dataset.reset_index()
       print(dataset.info())
       
       cols = ['LapTime', 'Sector1Time',
              'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2',
              'SpeedFL', 'SpeedST']

       RS = 20150101
       tsne = TSNE(random_state=RS).fit_transform(dataset[cols])

       classes = dataset['Driver'].astype('category').cat.codes
       scatter(tsne, classes,20,year_schedule['Location'][round])
       plt.show()

# Compute MDS 

In [4]:
import numpy as np
import pandas as pd
from sklearn.manifold import MDS
from sklearn import preprocessing
import fastf1

year = 2023
year_schedule = fastf1.get_event_schedule(year)
for round in year_schedule['RoundNumber'][1:]:
       race = fastf1.get_session(year, year_schedule['Location'][round], 'R')
       race.load()

       dataset = race.laps
       cols_to_remove =['Time', 'DriverNumber','PitOutTime','PitInTime', 'Sector1SessionTime', 
                        'Sector2SessionTime', 'Sector3SessionTime', 'IsPersonalBest',
                        'FreshTyre','LapStartTime','LapStartDate', 'Deleted',
                        'DeletedReason','FastF1Generated','IsAccurate']
       dataset = dataset.drop(cols_to_remove, axis = 1)
       dataset['TrackStatus'] = dataset['TrackStatus'].astype('float') #I hate this
       #drop rows where track status is not 1, i.e. the lap is not a 'racing' lap
       rows_to_remove = dataset.index[dataset['TrackStatus'] != 1].tolist()
       dataset = dataset.drop(rows_to_remove, axis = 0)


       #drop rows with missing data because i cant think of something better
       dataset = dataset.dropna(axis=0)
       #drop trackStatus as well after i used i to remove the rows i wanted
       dataset = dataset.drop('TrackStatus', axis = 1)
       #Convert timedelta64 to milliseconds
       cols_to_change = ['LapTime','Sector1Time',
              'Sector2Time', 'Sector3Time']
       dataset[cols_to_change] = dataset[cols_to_change].astype(np.int64)/ int(1e6)
       dataset =dataset.reset_index()
       print(dataset.info())
       
       cols = ['LapTime', 'Sector1Time',
              'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2',
              'SpeedFL', 'SpeedST']

       std_scale = preprocessing.StandardScaler().fit(dataset[cols])
       d = std_scale.transform(dataset[cols])
       mds = MDS(n_components=2, dissimilarity='euclidean', normalized_stress ="auto", random_state=None, max_iter=300)   
       proj_mds = mds.fit(dataset[cols]).embedding_

       classes = dataset['Driver'].astype('category').cat.codes
       scatter(proj_mds, classes,20,year_schedule['Location'][round])
       plt.show()

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data


KeyboardInterrupt: 

# PCA again


In [6]:
import fastf1
import pandas as pd
import os

#dim reduction preprocessing
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
import numpy as np

year = 2023
year_schedule = fastf1.get_event_schedule(year)
for round in year_schedule['RoundNumber'][1:]:
       race = fastf1.get_session(year, year_schedule['Location'][round], 'R')
       race.load()

       dataset = race.laps
       cols_to_remove =['DriverNumber','PitOutTime','PitInTime','IsPersonalBest','FreshTyre','LapStartTime', 'Position', 'Deleted','DeletedReason','FastF1Generated','IsAccurate']
       dataset = dataset.drop(cols_to_remove, axis = 1)
       dataset['TrackStatus'] = dataset['TrackStatus'].astype('float') #I hate this
       rows_to_remove = dataset.index[dataset['TrackStatus'] != 1].tolist()
       dataset = dataset.drop(rows_to_remove, axis = 0)


       #drop rows with missing data because i cant think of something better
       dataset = dataset.dropna(axis=0)
       #drop trackStatus as well after i used i to remove the rows i wanted
       dataset = dataset.drop('TrackStatus', axis = 1)
       #Convert timedelta64 to milliseconds
       cols_to_change = ['Time', 'LapTime','Sector1Time',
              'Sector2Time', 'Sector3Time', 'Sector1SessionTime',
              'Sector2SessionTime', 'Sector3SessionTime','LapStartDate']
       dataset[cols_to_change] = dataset[cols_to_change].astype(np.int64)/ int(1e6)
       dataset =dataset.reset_index()
       print(dataset.info())
       
       cols = ['LapTime', 'Sector1Time',
              'Sector2Time', 'Sector3Time','SpeedI1', 'SpeedI2',
              'SpeedFL', 'SpeedST']

       scaler= StandardScaler()
       scaled = scaler.fit_transform(dataset[cols])

       pca = PCA(n_components = 2)
       components = pca.fit_transform(scaled)
       principal_components = pca.components_
       df_components = pd.DataFrame(data = components, columns=['PC1','PC2'])

       df_components['Driver'] = dataset['Driver']
       df_components['Team'] = dataset['Team']
       df_components[cols] = dataset[cols]
       print(df_components)

       dirname = '../data/'+str(round)+'_'+year_schedule['Location'][round]
       df_components.to_csv(dirname+'/'+'PCA_2.csv')

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Pri

<class 'fastf1.core.Laps'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               599 non-null    int64  
 1   Time                599 non-null    float64
 2   Driver              599 non-null    object 
 3   LapTime             599 non-null    float64
 4   LapNumber           599 non-null    float64
 5   Stint               599 non-null    float64
 6   Sector1Time         599 non-null    float64
 7   Sector2Time         599 non-null    float64
 8   Sector3Time         599 non-null    float64
 9   Sector1SessionTime  599 non-null    float64
 10  Sector2SessionTime  599 non-null    float64
 11  Sector3SessionTime  599 non-null    float64
 12  SpeedI1             599 non-null    float64
 13  SpeedI2             599 non-null    float64
 14  SpeedFL             599 non-null    float64
 15  SpeedST             599 non-null    float64
 16  Compound           

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


<class 'fastf1.core.Laps'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               605 non-null    int64  
 1   Time                605 non-null    float64
 2   Driver              605 non-null    object 
 3   LapTime             605 non-null    float64
 4   LapNumber           605 non-null    float64
 5   Stint               605 non-null    float64
 6   Sector1Time         605 non-null    float64
 7   Sector2Time         605 non-null    float64
 8   Sector3Time         605 non-null    float64
 9   Sector1SessionTime  605 non-null    float64
 10  Sector2SessionTime  605 non-null    float64
 11  Sector3SessionTime  605 non-null    float64
 12  SpeedI1             605 non-null    float64
 13  SpeedI2             605 non-null    float64
 14  SpeedFL             605 non-null    float64
 15  SpeedST             605 non-null    float64
 16  Compound           

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '14', '18', '11', '4', '27', '81', '24', '22', '77', '55', '10', '31', '21', '2', '20', '63', '23', '16']
core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.2.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


...

<class 'fastf1.core.Laps'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               729 non-null    int64  
 1   Time                729 non-null    float64
 2   Driver              729 non-null    object 
 3   LapTime             729 non-null    float64
 4   LapNumber           729 non-null    float64
 5   Stint               729 non-null    float64
 6   Sector1Time         729 non-null    float64
 7   Sector2Time         729 non-null    float64
 8   Sector3Time         729 non-null    float64
 9   Sector1SessionTime  729 non-null    float64
 10  Sector2SessionTime  729 non-null    float64
 11  Sector3SessionTime  729 non-null    float64
 12  SpeedI1             729 non-null    float64
 13  SpeedI2             729 non-null    float64
 14  SpeedFL             729 non-null    float64
 15  SpeedST             729 non-null    float64
 16  Compound           

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
_api        WARNING 	Skipping lap alignment (no suit

<class 'fastf1.core.Laps'>
RangeIndex: 556 entries, 0 to 555
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               556 non-null    int64  
 1   Time                556 non-null    float64
 2   Driver              556 non-null    object 
 3   LapTime             556 non-null    float64
 4   LapNumber           556 non-null    float64
 5   Stint               556 non-null    float64
 6   Sector1Time         556 non-null    float64
 7   Sector2Time         556 non-null    float64
 8   Sector3Time         556 non-null    float64
 9   Sector1SessionTime  556 non-null    float64
 10  Sector2SessionTime  556 non-null    float64
 11  Sector3SessionTime  556 non-null    float64
 12  SpeedI1             556 non-null    float64
 13  SpeedI2             556 non-null    float64
 14  SpeedFL             556 non-null    float64
 15  SpeedST             556 non-null    float64
 16  Compound           

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	Using cached data for season_schedule
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.2.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Dat

<class 'fastf1.core.Laps'>
RangeIndex: 845 entries, 0 to 844
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               845 non-null    int64  
 1   Time                845 non-null    float64
 2   Driver              845 non-null    object 
 3   LapTime             845 non-null    float64
 4   LapNumber           845 non-null    float64
 5   Stint               845 non-null    float64
 6   Sector1Time         845 non-null    float64
 7   Sector2Time         845 non-null    float64
 8   Sector3Time         845 non-null    float64
 9   Sector1SessionTime  845 non-null    float64
 10  Sector2SessionTime  845 non-null    float64
 11  Sector3SessionTime  845 non-null    float64
 12  SpeedI1             845 non-null    float64
 13  SpeedI2             845 non-null    float64
 14  SpeedFL             845 non-null    float64
 15  SpeedST             845 non-null    float64
 16  Compound           